<a href="https://colab.research.google.com/github/artemiichirkov2/Unit2_Team8/blob/main/individual/Unit2_Artemii_Chirkov_BQML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Unit 2 — Team Classification (Titanic, BQML)

**Goal (team):** Build an *ops-ready* classifier in **BigQuery ML** to predict **`survived`** on the Titanic dataset. Requirements mirror the Flights notebook for comparability.
    
**Dataset:** `bigquery-public-data.ml_datasets.titanic`

**Deliver (inside this notebook):**
- One **LOGISTIC_REG** baseline + one **engineered** model (`TRANSFORM`)
- **Evaluation** via `ML.EVALUATE` and **confusion matrices** (0.5 + custom threshold)
- **Threshold choice** + 3–5 sentence ops justification (e.g., lifeboat allocation policy in a hypothetical ops setting)
- Embedded **rubric** below


**NOTE:** The table path for titanic dataset was not working. The error 404 says that the dataset does not exist. After asking the professor during the class and emailing the TA, we got the approval to download the dataset straight from Kaggle and use the data in our local environment.

**Source:** https://www.kaggle.com/competitions/titanic/data

In [ ]:
# --- CONFIG (uses your existing bq client) ---
PROJECT_ID  = "mgmt-467-47889"
SCHEMA      = f"{PROJECT_ID}.titanic"        # models will live here (same dataset as your tables)

TABLE_TRAIN = f"{SCHEMA}.train"
TABLE_TEST  = f"{SCHEMA}.test"
TABLE_GSUB  = f"{SCHEMA}.gender_submission"

# Policy knobs for later cells
CUSTOM_THRESHOLD = 0.75     # confusion @ custom threshold
C_FN, C_FP        = 4.0, 1.0  # false negative vs false positive costs for Model D

print("Models + tables dataset:", SCHEMA)
print("Train:", TABLE_TRAIN, "\nTest:", TABLE_TEST, "\nGender submission:", TABLE_GSUB)

Models + tables dataset: mgmt-467-47889.titanic
Train: mgmt-467-47889.titanic.train 
Test: mgmt-467-47889.titanic.test 
Gender submission: mgmt-467-47889.titanic.gender_submission


### Quick sanity check

In [ ]:

bq.query(f"SELECT * FROM `{TABLE_PATH}` LIMIT 5").result().to_dataframe()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,None,S
1,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S
2,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,None,S
3,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,None,S
4,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,None,S



## 1) Canonical mapping (minimal)
We map to:
- `survived` (BOOL), `pclass` (INT), `sex` (STRING), `age` (NUM), `sibsp` (INT), `parch` (INT), `fare` (NUM), `embarked` (STRING)


In [ ]:
# Maps to: survived (BOOL), pclass (INT), sex (STRING), age (NUM), sibsp (INT), parch (INT), fare (NUM), embarked (STRING)
CANONICAL_BASE_SQL = f'''
WITH titanic_c AS (
  SELECT
    CAST(PassengerId AS INT64)  AS passenger_id,
    CAST(Survived    AS BOOL)   AS survived,
    CAST(Pclass      AS INT64)  AS pclass,
    CAST(Sex         AS STRING) AS sex,
    CAST(Age         AS FLOAT64) AS age,
    CAST(SibSp       AS INT64)  AS sibsp,
    CAST(Parch       AS INT64)  AS parch,
    CAST(Fare        AS FLOAT64) AS fare,
    CAST(Embarked    AS STRING) AS embarked
  FROM `{TABLE_TRAIN}`
),
impute AS (
  SELECT
    passenger_id, survived, pclass, sex,
    COALESCE(age,  AVG(age)  OVER())        AS age,
    sibsp, parch,
    COALESCE(fare, AVG(fare) OVER())        AS fare,
    COALESCE(embarked, 'S')                  AS embarked
  FROM titanic_c
)
'''
print("Canonical SQL ready.")


Canonical SQL ready.


### 2) Split (80/20)

In [ ]:

SPLIT_CLAUSE = r'''
, split AS (
  SELECT
    i.*,
    CASE
      WHEN MOD(ABS(FARM_FINGERPRINT(CAST(passenger_id AS STRING))), 10) < 8
      THEN 'TRAIN' ELSE 'EVAL'
    END AS data_split
  FROM impute i
)
'''
print("Split clause: deterministic 80/20.")


Split clause: deterministic 80/20.



## 3) Baseline model — LOGISTIC_REG (`survived`)
Use a small set of signals (keep parity with Flights complexity).


In [ ]:
MODEL_BASE = f"{SCHEMA}.clf_survived_base"

sql_baseline = f"""
CREATE SCHEMA IF NOT EXISTS `{SCHEMA}`;

-- Train (manifest-only): pclass, sex, age, fare, embarked
CREATE OR REPLACE MODEL `{MODEL_BASE}`
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['survived']) AS
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}
SELECT survived, pclass, sex, age, fare, embarked
FROM split
WHERE data_split='TRAIN'
;

-- Evaluate on EVAL fold
SELECT * FROM ML.EVALUATE(
  MODEL `{MODEL_BASE}`,
  (
    {CANONICAL_BASE_SQL}
    {SPLIT_CLAUSE}
    SELECT survived, pclass, sex, age, fare, embarked
    FROM split
    WHERE data_split='EVAL'
  )
);
"""
bq.query(sql_baseline).result()
print("✅ Model A trained:", MODEL_BASE)


✅ Model A trained: mgmt-467-47889.titanic.clf_survived_base


### Confusion matrix — default 0.5 threshold

In [ ]:

cm_default_sql = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

, scored AS (
  SELECT
    t.survived AS label,
    p.predicted_survived AS pred_label,
    p.predicted_survived_probs[OFFSET(0)].prob AS score
  FROM split t
  JOIN ML.PREDICT(MODEL `{MODEL_BASE}`,
      (SELECT passenger_id, pclass, sex, age, sibsp, parch, fare, embarked FROM split WHERE data_split='EVAL')) AS p
  ON t.passenger_id = p.passenger_id
  WHERE t.data_split='EVAL'
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_default_sql).result().to_dataframe()


,TP,FP,FN,TN
0,42,16,15,107


### Confusion matrix — your custom threshold

In [ ]:

CUSTOM_THRESHOLD = 0.6   # TODO: justify in ops (e.g., conservative rescue policy)

cm_thresh_sql = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

, predictions AS (
    SELECT
        passenger_id,
        predicted_survived,
        predicted_survived_probs
    FROM ML.PREDICT(MODEL `{MODEL_BASE}`,
        (SELECT passenger_id, pclass, sex, age, sibsp, parch, fare, embarked FROM split WHERE data_split='EVAL'))
)
, scored AS (
  SELECT
    t.survived AS label,
    CAST(p.predicted_survived_probs[OFFSET(0)].prob >= {CUSTOM_THRESHOLD} AS BOOL) AS pred_label
  FROM split t
  JOIN predictions p
  ON t.passenger_id = p.passenger_id
  WHERE t.data_split='EVAL'
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_thresh_sql).result().to_dataframe()


,TP,FP,FN,TN
0,42,11,15,112



## 4) Engineered model — `TRANSFORM`
Create **family_size**, **fare_bucket**, and a **sex_pclass** interaction (categorical). Compare with baseline.


In [ ]:
MODEL_XFORM = f"{SCHEMA}.clf_survived_xform"

sql_xform = f"""
-- Train engineered model (adds family_size, fare_bucket, sex_pclass)
CREATE OR REPLACE MODEL `{MODEL_XFORM}`
TRANSFORM (
  (sibsp + parch + 1) AS family_size,
  CASE WHEN fare < 10 THEN 'low'
       WHEN fare < 50 THEN 'mid'
       ELSE 'high' END AS fare_bucket,
  CONCAT(sex, '_', CAST(pclass AS STRING)) AS sex_pclass,
  -- include manifest and label column
  survived, pclass, sex, age, sibsp, parch, fare, embarked
)
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['survived']) AS
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}
SELECT * FROM split WHERE data_split='TRAIN'
;

-- Compare A vs Engineered on EVAL fold
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}
SELECT 'baseline' AS model_version, * FROM ML.EVALUATE(
  MODEL `{MODEL_BASE}`,
  (SELECT survived, pclass, sex, age, sibsp, parch, fare, embarked
   FROM split WHERE data_split='EVAL')
)
UNION ALL
SELECT 'engineered' AS model_version, * FROM ML.EVALUATE(
  MODEL `{MODEL_XFORM}`,
  (SELECT * FROM split WHERE data_split='EVAL')
);
"""

# Execute the query and capture the results to display the comparison
eval_results = bq.query(sql_xform).result()
print("✅ Model A trained:", MODEL_XFORM)
print("\n--- Evaluation Comparison ---")
eval_results.to_dataframe()


✅ Model A trained: mgmt-467-47889.titanic.clf_survived_xform

--- Evaluation Comparison ---


,model_version,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,engineered,0.781818,0.754386,0.855556,0.767857,0.374878,0.874776
1,baseline,0.724138,0.736842,0.827778,0.730435,0.397937,0.865334



### Write-up (concise)
- **Threshold chosen & ops rationale:** …  
- **Baseline vs engineered — changes in AUC/precision/recall:** …  
- **Risk framing:** FP vs FN trade in a rescue/triage-like context: what error hurts more and why? …


**Threshold chosen & ops rationale:**

Used 0.60 for the classification cutoff (custom threshold cell). This slightly tightens the positive label vs. the default 0.50 and reduced false positives on the eval split (FP 16 → 11, TN 107 → 112) while leaving TP/FN unchanged in your run (TP 42, FN 15). Operationally: if follow-ups on predicted positives are costly/noisy, 0.60 improves precision without materially hurting recall in your eval sample. (Threshold and confusion tables are from the “custom threshold” section of the notebook.)

**Baseline vs engineered** — changes in AUC / precision / recall:
From the side-by-side ML.EVALUATE table:

AUC (ROC): 0.8653 → 0.8748 (Δ +0.0095)

Precision: 0.7241 → 0.7818 (Δ +0.0577)

Recall: 0.7368 → 0.7544 (Δ +0.0176)

(Models labeled “baseline” vs “engineered” in the comparison cell.)

Risk framing (FP vs. FN in rescue/triage):

False Negative (FN): Missing someone who actually needs help — the higher-stakes error in a rescue/triage context. FNs can lead to harm because no resources are deployed.

False Positive (FP): Mobilizing for someone who doesn’t actually need help — wastes resources but is usually recoverable.

**Which hurts more & why:**

FN is worse because it withholds intervention when it’s needed. If your downstream capacity allows, you’d normally bias the threshold down to raise recall and cut FNs, accepting more FPs. In your current eval snapshot, moving to 0.60 happened to trim FPs without increasing FNs, so it’s acceptable; but if future data show a recall drop, consider lowering the threshold (e.g., 0.45–0.50) or using cost-sensitive selection aligned to your C_FN > C_FP priority.


---

## Rubric (Titanic, 100 pts)
**Team-only deliverable in this notebook**

- Baseline LOGISTIC_REG + evaluation (AUC + confusion @0.5) — **20**  
- Custom threshold confusion matrix + ops justification — **20**  
- Engineered model with `TRANSFORM` (family_size, fare_bucket, sex_pclass) — **20**  
- Comparison table (baseline vs engineered) + 3–5 sentence interpretation — **20**  
- Reproducibility: parameters clear, no hidden magic; mapping documented — **10**  
- Governance notes: assumptions/limitations + slices you would monitor — **10**

> **Strictness:** No screenshots; use actual results cells. Keep explanations concise (bullet points OK).
